# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingGroundingTool, ListSortOrder, MessageRole
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
load_dotenv("azure.env")

True

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 17-Jun-2025 08:54:56


## Grounding Bing Search

In [4]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [5]:
# Shoud be: /subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.CognitiveServices/accounts/<ai_service_name>/projects/<project_name>/con#nections/<connection_name>
BING_CONN_ID = os.getenv("BING_CONN_ID")

In [7]:
bing = BingGroundingTool(connection_id=BING_CONN_ID)
bing

### Create agent with Bing Grounding tool

In [8]:
# Create the agent with the Bing grounding tool
agent = project_client.create_agent(
    model="gpt-4o",
    temperature=0.7,
    name="Bing-grounding-agent-demo",
    instructions="You are an AI Agent that can do some web search using Bing",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Agent with Grounding Bing Search",
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_xOhaLTsvIGe3LnPI1ZIFkWuh


### Example 1

In [9]:
prompt = "What is the current price of the MICROSOFT share in USD?"

In [10]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_3b1LW2BDRBsImACOpdXzUdxY
💬 [2] Created message | ID: msg_5QhcVatR7RwTRBT8QUfIVHDH
🏃 [3] Created run     | ID: run_fkpTIU0httNZay1eutIGMGwX
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_QRU9MjRrNuiPM4vMc5TKI4Zs, Type: message_creation
Details step: {'id': 'step_QRU9MjRrNuiPM4vMc5TKI4Zs', 'object': 'thread.run.step', 'created_at': 1750150530, 'run_id': 'run_fkpTIU0httNZay1eutIGMGwX', 'assistant_id': 'asst_xOhaLTsvIGe3LnPI1ZIFkWuh', 'thread_id': 'thread_3b1LW2BDRBsImACOpdXzUdxY', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150530, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_aK4jT1UibTqJGnCGIYjmmW3u'}}, 'usage': {'prompt_tokens': 5432, 'completion_tokens': 42, 'total_tokens': 5474, 'prompt_token_details': {'cached_tokens': 0}}}

🔍 Step ID 

### Example 2

In [13]:
prompt = "What are the news about the 2025 WWDC event?"

In [14]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")
    
# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_iaRtGVhm42ZoQa5Mkz1kUkTm
💬 [2] Created message | ID: msg_Ydx6RO3qQpH0LIBVKf1MdZ0n
🏃 [3] Created run     | ID: run_aVdQf0cgJScCwl3eGO08hMPa
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_yywG3zNt8rVFVUJtxLWBq5XT, Type: message_creation
Details step: {'id': 'step_yywG3zNt8rVFVUJtxLWBq5XT', 'object': 'thread.run.step', 'created_at': 1750150571, 'run_id': 'run_aVdQf0cgJScCwl3eGO08hMPa', 'assistant_id': 'asst_xOhaLTsvIGe3LnPI1ZIFkWuh', 'thread_id': 'thread_iaRtGVhm42ZoQa5Mkz1kUkTm', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150574, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_S3f9j5OewXBM0eIuaIGz05O6'}}, 'usage': {'prompt_tokens': 3730, 'completion_tokens': 355, 'total_tokens': 4085, 'prompt_token

### Example 3

In [15]:
prompt = "What are the weather informations for Paris? Show the report and use emojis and be funny"

In [16]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break
        
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_bYFks3XD13r5eGEXSgU2l8LS
💬 [2] Created message | ID: msg_K98KeCwXv5juLo8XhfHLZuRb
🏃 [3] Created run     | ID: run_t8LsjcUFOAWZGxB8wk9avwnD
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_TOdbqDwH1PeaeQ7WH5lJAwjR, Type: message_creation
Details step: {'id': 'step_TOdbqDwH1PeaeQ7WH5lJAwjR', 'object': 'thread.run.step', 'created_at': 1750150580, 'run_id': 'run_t8LsjcUFOAWZGxB8wk9avwnD', 'assistant_id': 'asst_xOhaLTsvIGe3LnPI1ZIFkWuh', 'thread_id': 'thread_bYFks3XD13r5eGEXSgU2l8LS', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150582, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_LZqy2AfuhmUBSFd9t8iO2NWo'}}, 'usage': {'prompt_tokens': 3857, 'completion_tokens': 136, 'total_tokens': 3993, 'prompt_token_details': {'cached_tokens': 0}}}

🔍 Step ID

### Example 4

In [17]:
prompt = "What can you tell about Roland Garros 2025?"

In [18]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_qaNTHsSqEiXOxl8Lw2qrQuPs
💬 [2] Created message | ID: msg_T2HYFcOE2qgpK2yTLAtOoNCm
🏃 [3] Created run     | ID: run_6S3unP2jmUHLaOQlW8p0VYqd
⏳ [4] Polling run status...
🔄 Status: queued
🔄 Status: queued
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_LQAuD354FKnw3zTuTAnhEROe, Type: message_creation
Details step: {'id': 'step_LQAuD354FKnw3zTuTAnhEROe', 'object': 'thread.run.step', 'created_at': 1750150590, 'run_id': 'run_6S3unP2jmUHLaOQlW8p0VYqd', 'assistant_id': 'asst_xOhaLTsvIGe3LnPI1ZIFkWuh', 'thread_id': 'thread_qaNTHsSqEiXOxl8Lw2qrQuPs', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150592, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_8zbqADQZl38yp46wyeWRlw7E'}}, 'usage': {'prompt_tokens': 3268, 'completion_tokens': 130, 'total_tokens': 3398, '

## Example 5

In [19]:
prompt = "Who won at the Cannes festival in 2025?"

In [20]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_sHjrcVEv6YXo1F8EplATG6Jb
💬 [2] Created message | ID: msg_L2FmLbplpGKhV5kx4hqlqkHx
🏃 [3] Created run     | ID: run_EtAmOfSF60jOJlETS5AXtylA
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_UWH6sr1p9BEfyap7SS67AdZi, Type: message_creation
Details step: {'id': 'step_UWH6sr1p9BEfyap7SS67AdZi', 'object': 'thread.run.step', 'created_at': 1750150597, 'run_id': 'run_EtAmOfSF60jOJlETS5AXtylA', 'assistant_id': 'asst_xOhaLTsvIGe3LnPI1ZIFkWuh', 'thread_id': 'thread_sHjrcVEv6YXo1F8EplATG6Jb', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150598, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_jjZLztXGDL25phuErauB7jU2'}}, 'usage': {'prompt_tokens': 3282, 'completion_tokens': 155, 'total_tokens': 3437, 'prompt_token_details': {'cached_tokens': 0}}}

🔍 Step ID

### Post processing

In [21]:
print(
    f"Agent Details:\n"
    f"📛 Name        : {agent.name}\n"
    f"🆔 ID          : {agent.id}\n"
    f"🧠 Model       : {agent.model}\n"
    f"📜 Instructions: {agent.instructions}"
)

Agent Details:
📛 Name        : Bing-grounding-agent-demo
🆔 ID          : asst_xOhaLTsvIGe3LnPI1ZIFkWuh
🧠 Model       : gpt-4o
📜 Instructions: You are an AI Agent that can do some web search using Bing


In [ ]:
project_client.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")